# Runtime
Calls the ML models, handles training, loads in the dataset, using all the other methods present in ./Experiments and ./Models

## Methods and Libraries

## Argument and Configuration Handler 

In [ ]:
# Determine MODEL to use
MODEL = 'null'
models = ['CDIL', 'DIL', 'TCN', 'CNN', 'Deformable', 'LSTM', 'GRU']
while MODEL not in models:
    print('Select one of the following models to train on the dataset: ', end='')
    for i in models:
        print(i, ' ', end='')
    MODEL = input('Selection:')


# Configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Using WANDB for visualization of validation and performance
wandb.init(project='DeFungi', name=MODEL, entity="jacob-davidian")
WANDB = wandb

# Config parameters
cfg_training = config['training']
cfg_model = config['models']

BATCH_SIZE = cfg_training['batch_size']
NUM_CLASSES = cfg_model['n_class']
SEQ_LEN = cfg_model['n_length']
FIX_LENGTH = cfg_model['fix_length']
USE_EMBED = cfg_model['use_embedding']
CHAR_COCAB = cfg_model['vocab_size']
INPUT_SIZE = cfg_model['dim']

LAYER = cfg_model['cnn_layer']
NHID = cfg_model['cnn_hidden']
KERNEL_SIZE = cfg_model['cnn_ks']

Select one of the following models to train on the dataset: CDIL  DIL  TCN  CNN  Deformable  LSTM  GRU  

Selection: CDIL


## Load and Prepare Dataset

In [ ]:
# Select between Grayscale or RGB dataloading
rgb = input('Please indicate if you would prefer RGB or Grayscale images. Enter 1 for RGB, 0 for Grayscale: ')

if(int(rgb)):
    train_loader, test_loader = DeFungiDataset('../EDA/Dataset', use_grayscale=False, batch_size=BATCH_SIZE, test_size=0.2)
else:
    train_loader, test_loader = DeFungiDataset('../EDA/Dataset', use_grayscale=True, batch_size=BATCH_SIZE, test_size=0.2)

## Model Selection

In [28]:
# Model selection with configs 
if MODEL == 'CDIL' or MODEL == 'DIL' or MODEL == 'TCN' or MODEL == 'CNN':
    LAYER = cfg_model['cnn_layer']
    NHID = cfg_model['cnn_hidden']
    KERNEL_SIZE = cfg_model['cnn_ks']
    net = CONV(MODEL, INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, False, False, USE_EMBED, CHAR_COCAB, FIX_length)
    receptive_field(seq_length=SEQ_LEN, model=MODEL, kernel_size=KERNEL_SIZE, layer=LAYER)
elif MODEL == 'Deformable':
    LAYER = cfg_model['cnn_layer']
    NHID = cfg_model['cnn_hidden']
    KERNEL_SIZE = cfg_model['cnn_ks']
    net = CONV('CNN', INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, True, False, USE_EMBED, CHAR_COCAB, FIX_length)
    receptive_field(seq_length=SEQ_LEN, model=MODEL, kernel_size=KERNEL_SIZE, layer=LAYER)
elif MODEL == 'LSTM' or MODEL == 'GRU':
    LAYER = cfg_model['rnn_layer']
    NHID = cfg_model['rnn_hidden']
    net = RNN(MODEL, INPUT_SIZE, CLASS, NHID, LAYER, USE_EMBED, CHAR_COCAB, FIX_length)

net = net.to(device)
para_num = sum(p.numel() for p in net.parameters() if p.requires_grad)

print(f'architecture of {MODEL}: \n{net}')

position: 512
layer:1 	 distance:3 	 size:3
layer:2 	 distance:7 	 size:7
layer:3 	 distance:15 	 size:15
layer:4 	 distance:31 	 size:31
layer:5 	 distance:63 	 size:63
layer:6 	 distance:127 	 size:127
layer:7 	 distance:255 	 size:255
layer:8 	 distance:511 	 size:511
layer:9 	 distance:1023 	 size:1023

Diagram of CNN: 
 CONV(
  (embedding): Embedding(256, 64)
  (conv): ConvPart(
    (conv_net): Sequential(
      (0): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (1): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (2): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (3): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), di

/home/hawkobo/anaconda3/envs/jupyterlab-debugger/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## Create Optimizer, Load Data (Train/Val/Test)

In [1]:
# Optimize
optimizer = torch.optim.Adam(net.parameters())
loss = torch.nn.CrossEntropyLoss(reduction='sum')


# Data
trainloader = DataLoader(DeFungiDataset(f'../defungi_encoded/image.train.pickle', True, label_encoder), batch_size=BATCH, shuffle=True, drop_last=False)
valloader = DataLoader(DeFungiDataset(f'../defungi_encoded/image.dev.pickle', True, label_encoder), batch_size=BATCH, shuffle=False, drop_last=False)
testloader = DataLoader(DeFungiDataset(f'../defungi_encoded/image.test.pickle', False, label_encoder), batch_size=BATCH, shuffle=False, drop_last=False)

NameError: name 'torch' is not defined

## Perform Training

In [ ]:
TrainModel(
    fix_length=FIX_length,
    net=net,
    device=device,
    trainloader=trainloader,
    valloader=valloader,
    testloader=testloader,
    n_epochs=cfg_training['epoch'],
    n_classes=cfg_model['n_class'],
    optimizer=optimizer,
    loss=loss,
    loginf=loginf,
    wandb=WANDB,
    file_name=model_name,
    label_encoder=label_encoder
)


  0%|                                                     | 0/1 [00:00<?, ?it/s]

## Pass to Front-End

In [ ]:
# Save Training Results for front-end
filename = 'fungi_classifier.sav'
pickle.dump(f'DeFungi_{MODEL}', open(filename, 'wb'))